In [1]:
import cv2

In [2]:
import numpy as np
import pandas as pd 
import os 

In [3]:
def KNN(k,train,test,label):
    
    distances = []
    cur_index = 0
    for cur in train:
        distances.append([euc_dist(test,cur),label[cur_index]])
        cur_index += 1
    
    distances = sorted(distances,key = lambda dist: dist[0],reverse=False)
    
    distances = distances[:k]
    distances = np.array(distances)
    # Frequency of unique classes, the one with the greater number is the ans,
    # k works better if odd (easier to choose).
    
    result = np.unique(distances[:,1],return_counts=True) # First column contains classes (hence for all rows 1st column) 
    # This comes to us in the sorted order of frequencies, hence we just need to return the last element 
    # result is in the form of [[values], [frequencies]]
    return result[0][-1] # max frequencied class whose points are the clossest to the given point   

In [4]:
def euc_dist(A,B):
    '''
    Here, we receive 2 points A and B (as numpy arrays),
    which may represent points marked over a n - dimensional plot.
    i.e, A = (x_i, y_i, z_i,...,n_i) and B = (x_j, y_j, z_j,...,n_j). 
    
    But here,
    A = (x_i, y_i) and B = (x_j, y_j)
    '''
    return np.sqrt(sum((A-B)**2))

In [5]:
# Retrieval from database

import mysql.connector as mc

In [6]:
cnx = mc.connect(host = 'localhost', username = 'vikranttandon', password = '******')

In [7]:
cursor = cnx.cursor(buffered= True)

In [8]:
cursor.execute('use dataset;')

In [9]:
cursor.execute('select * from image_frames;')

In [10]:
all_images = cursor.fetchall()

In [11]:
cursor.execute('select * from profile;')

In [12]:
all_labels = cursor.fetchall()

In [13]:
print(len(all_images),len(all_labels))

105 4


In [140]:
face_dataset = []
labels = []

ranges = []

Labeling :
1) Mainting a list of labels extracted from the database.
<br>2) Making numeric_labels based on the size of image frames for a single label.

In [141]:
# 1
for cur in all_labels:
    labels.append(cur[1])
    ranges.append((cur[2],cur[3]))

In [142]:
# 2
face_labels = []
cnt = 0
for start,end in ranges:
    size = end-start
    face_labels.append(cnt*np.ones(size))
    cnt += 1

In [ ]:
'''
Had to make dtype as int64 to receive the correct amount of data
'''

In [143]:
for cur in all_images:
    cur_face = cur[1]
    cur_face = np.frombuffer(cur_face,dtype = 'int64')
    face_dataset.append(cur_face)

In [144]:
face_dataset = np.concatenate(face_dataset,axis = 0)

In [145]:
face_dataset = np.reshape(face_dataset,(len(all_images),-1))

In [146]:
face_labels = np.concatenate(face_labels,axis = 0)

In [157]:
violators = {}

In [158]:
names = {}
for idx in range(len(labels)):
    names[idx] = labels[idx]

In [159]:
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")  

while True:
    ret,frame = cap.read()
    if ret == False:
        continue
        
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame,1.3,3)
    
    if len(faces) == 0:
        continue
        
    faces = sorted(faces,key=lambda cur_face: cur_face[2]*cur_face[3],reverse=True)
    face = faces[0]
    x,y,w,h = face
        
    cv2.rectangle(gray_frame,(x,y),(x+w,y+h),(255,255,255),3)
        
    roi_frame = gray_frame[y:y+h , x:x+w]
    roi_frame = cv2.resize(roi_frame,(200,200))
    
    test = np.asarray(roi_frame).reshape((-1,))
    output = KNN(5,face_dataset,test,face_labels)
    
    if names[output] not in violators:
        violators[names[output]] = 1 
        
    cv2.putText(gray_frame,names[output],(x,y-10),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    
    cv2.imshow("gray",gray_frame)
        
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('s'):
        break
            
cap.release()
cv2.destroyAllWindows()

In [160]:
from datetime import date
today = date.today()

In [161]:
with open('VIOLATORS','w+') as f:
    f.seek(0)
    f.write('VIOLATORS FOR ' + str(today)+'\n')
    
    cnt = 1
    for cur in violators.keys():
        f.write(str(cnt) + '.' + str(cur) +'\n')
        cnt += 1